In [1]:
import os 
import pandas as pd
import numpy as np

# import sklearn methods 
from sklearn.metrics import accuracy_score, roc_curve, confusion_matrix, classification_report, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import GridSearchCV
import sys 
from sklearn.model_selection import learning_curve
from sklearn.datasets import make_classification
import lightgbm as lgb

In [2]:
# display test scores and return result string and indexes of false samples
def display_test_scores(test, pred):
    str_out = ""
    str_out += ("TEST SCORES\n")
    str_out += ("\n")

    #print accuracy
    accuracy = accuracy_score(test, pred)
    str_out += ("ACCURACY: {:.4f}\n".format(accuracy))
    str_out += ("\n")

    #print AUC score
    auc = roc_auc_score(test, pred)
    str_out += ("AUC: {:.4f}\n".format(auc))
    str_out += ("\n")

    #print confusion matrix
    str_out += ("CONFUSION MATRIX:\n")
    conf_mat = confusion_matrix(test, pred)
    str_out += ("{}".format(conf_mat))
    str_out += ("\n")
    str_out += ("\n")

    #print FP, FN
    str_out += ("FALSE POSITIVES:\n")
    fp = conf_mat[1][0]
    pos_labels = conf_mat[1][0]+conf_mat[1][1]
    str_out += ("{} out of {} positive labels ({:.4f}%)\n".format(fp, pos_labels,fp/pos_labels))
    str_out += ("\n")

    str_out += ("FALSE NEGATIVES:\n")
    fn = conf_mat[0][1]
    neg_labels = conf_mat[0][1]+conf_mat[0][0]
    str_out += ("{} out of {} negative labels ({:.4f}%)\n".format(fn, neg_labels, fn/neg_labels))
    str_out += ("\n")

    #print classification report
    str_out += ("PRECISION, RECALL, F1 scores:\n")
    str_out += ("{}".format(classification_report(test, pred)))
    
    false_indexes = np.where(test != pred)
    return str_out, false_indexes

In [3]:
################ not scaled dataset ###################
df_original = pd.read_pickle('../input/ddd-final-datasets/final_step2.pkl')

X = df_original.loc[:, ["n_EAR", 
                    "n_MAR", "n_MOE", "n_EC",
                    "n_LEB", "n_SOP", "PERCLOS", "CLOSENESS"]]

y = df_original.loc[:, "DROWSINESS"]


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, stratify = y)

In [4]:
################ scaled dataset ###################
df_original2 = pd.read_pickle('../input/ddd-final-datasets/final_step2_scaled.pkl')

X2 = df_original2.loc[:, ["n_EAR", 
                    "n_MAR", "n_MOE", "n_EC",
                    "n_LEB", "n_SOP", "PERCLOS", "CLOSENESS"]]

y2 = df_original2.loc[:, "DROWSINESS"]


X_train2, X_test2, y_train2, y_test2 = train_test_split(X2, y2, test_size = 0.2, stratify = y2)

In [5]:
# cross-validation with 5 splits
cv = StratifiedShuffleSplit(n_splits=10, random_state = 42)

# CART decision tree
lgb_estimator = lgb.LGBMClassifier(random_state=42, bjective='binary', metric='auc', device= "gpu")

# parameters 
parameters = {
            'reg_lambda': [0.1, 1, 10],
            'n_estimators': [100, 500],
            'max_depth': [5, 10],
            'learning_rate':[0.001, 0.01, 0.1]
}

In [6]:
# grid search for parameters
grid = GridSearchCV(estimator=lgb_estimator, param_grid=parameters, cv=cv, n_jobs=-1)
grid.fit(X_train, y_train)

# print best scores
print("The best parameters are %s with a score of %0.4f"
      % (grid.best_params_, grid.best_score_))

# prediction results
y_pred = grid.predict(X_test)

# print accuracy metrics
results, false = display_test_scores(y_test, y_pred)
print(results)

The best parameters are {'learning_rate': 0.1, 'max_depth': 10, 'n_estimators': 500, 'reg_lambda': 0.1} with a score of 0.7510
TEST SCORES

ACCURACY: 0.7494

AUC: 0.7344

CONFUSION MATRIX:
[[32897 17928]
 [12873 59221]]

FALSE POSITIVES:
12873 out of 72094 positive labels (0.1786%)

FALSE NEGATIVES:
17928 out of 50825 negative labels (0.3527%)

PRECISION, RECALL, F1 scores:
              precision    recall  f1-score   support

           0       0.72      0.65      0.68     50825
           1       0.77      0.82      0.79     72094

    accuracy                           0.75    122919
   macro avg       0.74      0.73      0.74    122919
weighted avg       0.75      0.75      0.75    122919



fit for scaled dataset

In [7]:
# grid search for parameters
grid2 = GridSearchCV(estimator=lgb_estimator, param_grid=parameters, cv=cv, n_jobs=-1)
grid2.fit(X_train2, y_train2)

# print best scores
print("The best parameters are %s with a score of %0.4f"
      % (grid2.best_params_, grid2.best_score_))

# prediction results
y_pred2 = grid2.predict(X_test2)

# print accuracy metrics
results2, false2 = display_test_scores(y_test2, y_pred2)
print(results2)

The best parameters are {'learning_rate': 0.1, 'max_depth': 10, 'n_estimators': 500, 'reg_lambda': 1} with a score of 0.7489
TEST SCORES

ACCURACY: 0.7513

AUC: 0.7355

CONFUSION MATRIX:
[[32736 18089]
 [12485 59609]]

FALSE POSITIVES:
12485 out of 72094 positive labels (0.1732%)

FALSE NEGATIVES:
18089 out of 50825 negative labels (0.3559%)

PRECISION, RECALL, F1 scores:
              precision    recall  f1-score   support

           0       0.72      0.64      0.68     50825
           1       0.77      0.83      0.80     72094

    accuracy                           0.75    122919
   macro avg       0.75      0.74      0.74    122919
weighted avg       0.75      0.75      0.75    122919

